In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lmfit import Model

# Introduction

This notebook aims to model my own estradiol serum concentration from blood test results over the past few months.

We start with a 3-compartment linear pharmacokinetic model, then derive the theoretical serum estradiol concentration from the model for completeness (as well as for the learning experience in solving differential equations using Laplace transform).

[An Informal Meta-Analysis of Estradiol Curves with Injectable Estradiol Preparations](https://transfemscience.org/articles/injectable-e2-meta-analysis/) by Aly from Transfeminine Science included a model of serum estradiol for a given injectable estradiol regimen. However, I'm currently using various strengths of estradiol patches (a mixture of Estradot and Mylan Estradiol TDP).

The aforementioned article from Transfeminine Science uses compartment models obtained from the literature. Here, we will also use a compartment model, in particular the 3-compartment model.

# Three Compartment Model

The compartment models have the form of the [Bateman equation](https://en.wikipedia.org/wiki/Bateman_equation) that's used in physics to describe nuclear decay rate by modeling the transition rates from one species to another. It's also an example of a classical [master equation](https://en.wikipedia.org/wiki/Master_equation) where we only have a diagonal matrix that describes the "transition" probabilities (a weird word choice here due to my first encounter with master equations come from quantum optics).

The dynamics of a simple 3-compartment model is given by a system of linear differential equations ([Sheiner et al., 1979](https://doi.org/10.1002/cpt1979253358))

\begin{align*}
    \dot D(t) &= -k_1 D(t) \\
    \dot A_{\rm eff}(t) &= k_1 D(t) - k_2 A_{\rm eff}(t) \\
    \dot E_2(t) &= k_2 A_{\rm eff}(t) - k_3 E_2(t),
\end{align*}

where $\dot{}$ denotes time derivatives, $D(t)$ is the estradiol concentration within the depot (e.g., the patch sites at $200\rm~\mu g/day$), $A_{\rm eff}$ is the (hypothetical) amount of estradiol in the effect compartment, and $E_2(t)$ is the observed serum estradiol concentration at time $t$.

To solve the above system, we use Laplace transform to first annihilate the time derivatives and turn the system into a system of algebraic equations. The Laplace transform of first-order derivative $\dot f(t)$ of the function $f(t)$ is given by $$\mathcal{L}[\dot f(t)]=s\hat f(s)-f(t_0),$$ and Laplace transform is linear.

Thus, our system of differential equations in the Laplace space $s$ is

\begin{align*}
    s\hat D(s)-D(t_0) &= -k_1\hat D(s) \\
    s\hat A_{\rm eff}(s)-A_{\rm eff}(t_0) &= k_1\hat D(s)-k_2\hat A_{\rm eff}(s)\\
    s\hat E_2(s)-E_2(t_0) &= k_2\hat A_{\rm eff}(s) - k_3\hat E_2(s).
\end{align*}

After grouping like terms, we can rewrite the above system in matrix notation $$\begin{bmatrix}s+k_1 & 0 & 0 \\ -k_1 & s+k_2 & 0 \\ 0 & -k_2 & s+k_3\end{bmatrix}\begin{bmatrix}\hat D(s) \\ \hat A_{\rm eff}(s) \\ \hat E_2(s)\end{bmatrix}=\begin{bmatrix}D(t_0) \\ A_{\rm eff}(t_0) \\ E_2(t_0) \end{bmatrix}.$$

Next, to solve this algebraic system in Laplace space $s$, we use [Cramer's rule](https://en.wikipedia.org/wiki/Cramer's_rule), giving us

\begin{align*}
    \hat D(s)&=\frac{\operatorname{det}\left|\begin{matrix}D(t_0) & 0 & 0 \\ A_{\rm eff}(t_0) & s+k_2 & 0 \\ E_2(t_0) & -k_2 & s+k_3\end{matrix}\right|}{\operatorname{det}\left|\begin{matrix}s+k_1 & 0 & 0 \\ -k_1 & s+k_2 & 0 \\ 0 & -k_2 & s+k_3\end{matrix}\right|}=\frac{D(t_0)}{s+k_1} \\ \\
    \hat A_{\rm eff} &=\frac{\operatorname{det}\left|\begin{matrix}s+k_1 & D(t_0) & 0 \\ -k_1 & A_{\rm eff}(t_0) & 0 \\ 0 & E_2(t_0) & s+k_3\end{matrix}\right|}{\operatorname{det}\left|\begin{matrix}s+k_1 & 0 & 0 \\ -k_1 & s+k_2 & 0 \\ 0 & -k_2 & s+k_3\end{matrix}\right|}=\frac{k_1D(t_0)}{(s+k_1)(s+k_2)}+\frac{A_{\rm eff}(t_0)}{s+k_2} \\ \\
    \hat E_2(s)&=\frac{\operatorname{det}\left|\begin{matrix}s+k_1 & 0 & D(t_0) \\ -k_1 & s+k_2 & A_{\rm eff}(t_0) \\ 0 & -k_2 & E_2(t_0)\end{matrix}\right|}{\operatorname{det}\left|\begin{matrix}s+k_1 & 0 & 0 \\ -k_1 & s+k_2 & 0 \\ 0 & -k_2 & s+k_3\end{matrix}\right|}=\frac{k_1k_2D(t_0)}{(s+k_1)(s+k_2)(s+k_3)}+\frac{k_2A_{\rm eff}}{(s+k_2)(s+k_3)}+\frac{E_2(t_0)}{s+k_3}.
\end{align*}

$\hat E_2(s)$ is our solution to serum estradiol concentration that we can use to model the observed serum level obtained from blood tests. However, it's still in Laplace space $s$. To convert back to the time domain solution $E_2(t)$, we use [an inverse Laplace transform lookup table](https://tutorial.math.lamar.edu/Classes/DE/Laplace_Table.aspx) (as well as Wolfram|Alpha when the denominator contains a quadratic or higher-order polynomial in $s$), we thus find that

\begin{align*}
    E_2(t) = &~k_1k_2D(t_0)\left[\frac{e^{-k_1t}}{(k_1-k_2)(k_1-k_3)} - \frac{e^{-k_2t}}{(k_1-k_2)(k_2-k_3)} + \frac{e^{-k_3t}}{(k_1-k_3)(k_2-k_3)}\right] \\
    &~k_2A_{\rm eff}(t_0)\left[-\frac{e^{-k_2t}}{k_2-k_3} + \frac{e^{-k_3t}}{k_2-k_3}\right] + E_2(t_0)e^{-k_3t},
\end{align*} which matches with the solution found on [Reddit](https://old.reddit.com/r/TransDIY/comments/18zd4qw/routine_change_to_imitate_menstrual_cycle_levels/kghcqhy/).

To model estradiol serum concentration over repeated application of estradiol patches, we consider the following, as described [here](https://old.reddit.com/r/TransDIY/comments/18zd4qw/routine_change_to_imitate_menstrual_cycle_levels/kghcqhy/):

\begin{align*}
    E_2(t) = &~k_1k_2\sum_iD_i(t_0)H(t-t_i)\left[\frac{e^{-k_1(t-t_i)}}{(k_1-k_2)(k_1-k_3)} - \frac{e^{-k_2(t-t_i)}}{(k_1-k_2)(k_2-k_3)} + \frac{e^{-k_3(t-t_i)}}{(k_1-k_3)(k_2-k_3)}\right] \\
    &~k_2A_{\rm eff}(t_0)\left[-\frac{e^{-k_2t}}{k_2-k_3} + \frac{e^{-k_3t}}{k_2-k_3}\right] + E_2(t_0)e^{-k_3t},
\end{align*} where $t_i$ is the application time of a new patch, $D_i(t_0)$ is the total (initial) estradiol concentration of patches in a cycle, and $H(t-t_i)$ is the Heaviside function so that we can model the serum concentration over multiple patches cycles.

# Fitting Literature Release Profiles

Estradiol patches have a non-constant release profile. For example, Mylan Estradiol TDP's mean serum estradiol concentration of $n=36$ patients from a single dose of $100\rm~\mu g$ stays at the maximum only for the first 1.5 days ([DailyMed, 2022](https://dailymed.nlm.nih.gov/dailymed/drugInfo.cfm?setid=f4efb0bf-b96f-4e20-a0f2-0806ab92b2d4)):

![](https://dailymed.nlm.nih.gov/dailymed/image.cfm?name=export-09.jpg&setid=f4efb0bf-b96f-4e20-a0f2-0806ab92b2d4)

The mean serum estradiol from 4 doses of Estradot patches (sample size of $n=30$) similarly fluctuates over time [(Hossain et al., 2003a)](https://doi.org/10.1016/S0378-5122(03)00189-0):

![](https://ars.els-cdn.com/content/image/1-s2.0-S0378512203001890-gr1.gif)

The above study's authors also published another study containing data on mean serum estradiol of various sample sizes at two different sample sizes ($n_1=30,n_2=11$) is also included in our dataset [(Hossain et al., 2003b)](https://doi.org/10.1016/S0378-5122(03)00190-7). Sizes of the Estradot patches from these two studies are still applicable to date [(Ankarberg-Lindgren et al., 2019)](https://doi.org/10.1530/EC-19-0025).

While our simple model here only provides a simple decaying exponential for the depot, we have enough degrees of freedom from the effect compartment and measured level compartment to capture the actual release profile (i.e., the fitting parameter of the effect compartment $k_2$ as well as the level compartment $k_3$).

We will extract data from the figures of the above studies using [WebPlotDigitizer](https://automeris.io/WebPlotDigitizer/).